In [1]:
#Importing dependencies
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_log_error
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
%matplotlib inline
import itertools
import warnings
plt.style.use('ggplot')

In [16]:
#Loading Data
data = pd.read_csv('C:/Pythondatasets/All_Valid_Datasets_Sarajevo_vol2.csv')
data=data.drop_duplicates()
data

,Site,Parameter,Date (LT),Year,Month,Day,Hour,NowCast Conc.,AQI,AQI Category,Raw Conc.,Conc. Unit,Duration,QC Name
0,Sarajevo,PM2.5 - Principal,2018-05-22 05:00 PM,2018,5,22,17,10.0,42,Good,7.0,UG/M3,1 Hr,Valid
1,Sarajevo,PM2.5 - Principal,2018-05-22 06:00 PM,2018,5,22,18,15.7,59,Moderate,20.0,UG/M3,1 Hr,Valid
2,Sarajevo,PM2.5 - Principal,2018-05-22 07:00 PM,2018,5,22,19,14.2,55,Moderate,13.0,UG/M3,1 Hr,Valid
3,Sarajevo,PM2.5 - Principal,2018-05-22 08:00 PM,2018,5,22,20,16.7,61,Moderate,19.0,UG/M3,1 Hr,Valid
4,Sarajevo,PM2.5 - Principal,2018-05-22 09:00 PM,2018,5,22,21,13.8,55,Moderate,11.0,UG/M3,1 Hr,Valid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45178,Sarajevo,PM2.5 - Principal,2023-04-30 09:00 PM,2023,4,30,21,15.2,58,Moderate,21.0,UG/M3,1 Hr,Valid
45179,Sarajevo,PM2.5 - Principal,2023-04-30 10:00 PM,2023,4,30,22,23.6,75,Moderate,32.0,UG/M3,1 Hr,Valid
45180,Sarajevo,PM2.5 - Principal,2023-04-30 11:00 PM,2023,4,30,23,23.8,76,Moderate,24.0,UG/M3,1 Hr,Valid
45181,Sarajevo,PM2.5 - Principal,2023-05-01 12:00 AM,2023,5,1,0,25.4,79,Moderate,27.0,UG/M3,1 Hr,Valid


In [21]:
data['Date (LT)'] = pd.to_datetime(data['Date (LT)'])
data = data.set_index('Date (LT)')

data

KeyError: 'Date (LT)'

In [23]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(data['AQI'], model='multiplicative')

ValueError: You must specify a period or x must be a pandas object with a PeriodIndex or a DatetimeIndex with a freq not set to None

In [24]:
dataAQI = data[['AQI']].copy()
dataAQI

,AQI
Date (LT),
2018-05-22 17:00:00,42
2018-05-22 18:00:00,59
2018-05-22 19:00:00,55
2018-05-22 20:00:00,61
2018-05-22 21:00:00,55
...,...
2023-04-30 21:00:00,58
2023-04-30 22:00:00,75
2023-04-30 23:00:00,76


In [25]:
# We'll use statsmodels to perform a decomposition of this time series
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(dataAQI, model='multiplicative')

fig = result.plot()

ValueError: You must specify a period or x must be a pandas object with a PeriodIndex or a DatetimeIndex with a freq not set to None

In [26]:
!pip install pmdarima

In [27]:
!pip install --upgrade scipy


     ---------------------------------------- 44.1/44.1 MB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 15.1/15.1 MB 3.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: scipy
    Found existing installation: scipy 1.9.1
    Uninstalling scipy-1.9.1:
      Successfully uninstalled scipy-1.9.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.25.1 which is incompatible.


In [28]:
# The Pmdarima library for Python allows us to quickly perform this grid search 
from pmdarima import auto_arima

In [29]:
stepwise_model = auto_arima(dataAQI, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,1)[12]             : AIC=inf, Time=67.13 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=345630.762, Time=1.67 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=325972.982, Time=11.93 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=23.01 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=339715.757, Time=3.65 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=321732.745, Time=33.09 sec
 ARIMA(1,1,0)(2,1,1)[12]             : AIC=inf, Time=213.00 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=inf, Time=20.87 sec
 ARIMA(0,1,0)(2,1,0)[12]             : AIC=326175.204, Time=48.37 sec


MemoryError: Unable to allocate 481. MiB for an array with shape (41477, 39, 39) and data type float64